In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt

def generate_frog_call(fs=44100, flagtesttype=0, flagpulsereverse=0):
    """
    Generate synthetic gray tree frog call
    
    Parameters:
    fs : int
        Sampling frequency in Hz
    flagtesttype : int
        Test type flag that modifies various parameters
    flagpulsereverse : int
        Flag to reverse the pulse if set to 1
        
    Returns:
    tuple : (frogfake, envelopefake) - The synthetic call and its envelope
    """
    # SETUP
    starttimefakepulse = 0.0
    peaktimefakepulse = starttimefakepulse + 0.006
    endtimefakepulse = peaktimefakepulse + 0.006
    
    # Adjust timing parameters based on flagtesttype
    if flagtesttype in [10, 11, 12, 13, 14, 15]:
        peaktimefakepulse = starttimefakepulse + 0.0001
        endtimefakepulse = peaktimefakepulse + 0.012
        
    if flagtesttype in [16, 17, 18]:
        peaktimefakepulse = starttimefakepulse + 0.006
        endtimefakepulse = peaktimefakepulse + 0.024
        
    peakampfakepulse = 1.0
    endampfakepulse = 0.0
    
    # Set frequency
    ffake = 4000  # 2147 is the standard frequency used in Study 1
    if flagtesttype in [22, 23, 24]:
        ffake = 700  # Drop to one third for head size adjustment
        
    starttimefake = 0
    peaktimefake = starttimefake + 0.22
    
    if flagtesttype in [-1, -6]:
        peaktimefake = starttimefake + 0.0001
        
    pulseintervalfake = 0.300/7
    pulsect = 11
    endtimefake = starttimefake + pulsect * pulseintervalfake
    
    peakampfake = 0.028
    endampfake = peakampfake
    
    if flagtesttype in [19, 20, 21]:
        pulsect = 2 * pulsect
        endtimefake = starttimefake + pulsect * pulseintervalfake
        
    # Convert time to samples
    starttimefakepulsesamp = int(np.ceil(starttimefakepulse * fs)) + 1
    peaktimefakepulsesamp = int(np.round(peaktimefakepulse * fs))
    endtimefakepulsesamp = int(np.floor(endtimefakepulse * fs))
    
    starttimefakesamp = int(np.ceil(starttimefake * fs)) + 1
    peaktimefakesamp = int(np.round(peaktimefake * fs))
    endtimefakesamp = int(np.floor(endtimefake * fs))
    
    pulseintervalfakesamp = int(np.round(pulseintervalfake * fs))
    
    # MAKE ENVELOPE
    envelopefake = np.zeros(endtimefakesamp)
    
    # Generate rising part of envelope
    t_rise = np.arange(1, peaktimefakesamp - starttimefakesamp + 2)
    envelopefake[starttimefakesamp-1:peaktimefakesamp] = (
        peakampfake * np.sin(0.5 * np.pi * t_rise / (peaktimefakesamp - starttimefakesamp + 1))
    )
    
    # Generate falling part of envelope
    tcfake = -(endtimefakesamp - peaktimefakesamp) / np.log(endampfake / (1.001 * peakampfake))
    t_fall = np.arange(1, endtimefakesamp - peaktimefakesamp + 1)
    envelopefake[peaktimefakesamp:endtimefakesamp] = (
        peakampfake * np.exp(-t_fall / tcfake)
    )
    
    # Generate pulse envelope
    envelopefakepulse = np.zeros(endtimefakepulsesamp)
    
    # Rising part of pulse
    t_pulse_rise = np.arange(1, peaktimefakepulsesamp - starttimefakepulsesamp + 2)
    envelopefakepulse[starttimefakepulsesamp-1:peaktimefakepulsesamp] = (
        peakampfakepulse * np.sin(0.5 * np.pi * t_pulse_rise / (peaktimefakepulsesamp - starttimefakepulsesamp + 1))
    )
    
    # Falling part of pulse
    t_pulse_fall = np.arange(1, endtimefakepulsesamp - peaktimefakepulsesamp + 1)
    envelopefakepulse[peaktimefakepulsesamp:endtimefakepulsesamp] = (
        peakampfakepulse * np.cos(0.5 * np.pi * t_pulse_fall / (endtimefakepulsesamp - peaktimefakepulsesamp))
    )
    
    # Generate carrier wave
    phasefrogfakepulse = np.zeros(endtimefakepulsesamp)
    t_carrier = np.arange(1, endtimefakepulsesamp - starttimefakepulsesamp + 2)
    phasefrogfakepulse[starttimefakepulsesamp-1:endtimefakepulsesamp] = (
        np.sin(2 * np.pi * (ffake/fs) * t_carrier)
    )
    
    # Combine envelope and carrier
    frogfakepulse = envelopefakepulse * phasefrogfakepulse
    
    # Handle special test cases
    if flagtesttype in [4, 5, 6, 7, 8, 9, -1]:
        if flagtesttype == 4:
            fakecarrier0 = np.random.randn(len(frogfakepulse))
        if flagtesttype == 7:
            fakecarrier0 = np.random.randn(len(frogfakepulse))
            # Note: makeiirbank and filteritbank functions would need to be implemented
            # [fasiir, fbsiir, fcenters] = makeiirbank(fs, ffake, ffake, 12, 6)
            # fakecarrier0 = filteritbank(fakecarrier0, fasiir, fbsiir, 2)
        
        powerfakecarrier = np.var(fakecarrier0)
        fakecarrier = (1/np.sqrt(2 * powerfakecarrier)) * fakecarrier0
        frogfakepulse = envelopefakepulse * fakecarrier
    
    # Handle pulse reversal
    if flagpulsereverse or flagtesttype in [13, 14, 15]:
        frogfakepulse = frogfakepulse[::-1]
    
    # ASSEMBLE CALL
    frogfake = np.zeros_like(envelopefake)
    
    # Add pulses
    for ii in range(pulsect):
        start_idx = starttimefakesamp + ii * pulseintervalfakesamp
        end_idx = start_idx + endtimefakepulsesamp - 1
        if end_idx <= len(frogfake):
            frogfake[start_idx-1:end_idx] = frogfakepulse
    
    # Apply overall envelope
    frogfake = envelopefake * frogfake
    
    return frogfake, envelopefake

def plot_frog_call(frogfake, envelopefake):
    """Plot the generated frog call and its envelope"""
    plt.figure(figsize=(10, 6))
    plt.plot(envelopefake, label='Envelope')
    plt.plot(frogfake, label='Frog Call')
    plt.xlabel('Time (samples)')
    plt.ylabel('Signal')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage
if __name__ == "__main__":
    fs = 44100  # Sample rate in Hz
    frogfake, envelopefake = generate_frog_call(fs)
    plot_frog_call(frogfake, envelopefake)

In [3]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
import slab

In [4]:
fs = 44100
level = 70
slab.set_default_level(70)
slab.set_default_samplerate(44100)

In [5]:
def create_Binaural(sound):
    return slab.Binaural(sound)

In [6]:
def plot_itd(link):
    sound = slab.Binaural(link)
    sound.waveform()

In [7]:
def get_angled_sound(angle, sound, externalise, frequency):
    """
        This function takes in azimuth angle and monoaural sound and returns sound with the specified angle.

        Angle : azimuth angle. Where 0 is the center. So one posituve integer.
        Sound : the sound on which you want to get the angled sound.
        externalise : Boolean. If the function should externalise sound or not.
        frequency : Frequency of the sound to calculate ITD.
        head_size : Optional. In case you want calculate ITD specific to head sizes.

        Returns : Two slab.Binaural instances with left and right versions of the sound.
    """

    # calculating ITD
    itd = sound.azimuth_to_itd(angle, frequency = frequency)
    
    # applying itd to sound
    right = sound.itd(duration = itd)
    left = sound.itd(duration = -itd)

    if externalise:
        right = right.externalize()
        left = left.externalize()
    
    right.level = level
    left.level = level

    return left, right

In [8]:
def make_frog_call(fs=44100, frequency = 4000):
    """
    Generates a single-channel frog call without spatial/ITD cues.
    Based on the gray tree frog call implementation but simplified for monaural output.
    """
    # Initialize parameters exactly as in original
    start_time_pulse = 0.0
    peak_time_pulse = start_time_pulse + 0.006
    end_time_pulse = peak_time_pulse + 0.006
    
    peak_amp_pulse = 1.0
    end_amp_pulse = 0.0
    
    f_fake = frequency
    
    start_time = 0
    peak_time = start_time + 0.22
    pulse_interval = 0.300/7
    pulse_count = 11
    end_time = start_time + pulse_count * pulse_interval
    
    peak_amp = 0.028
    end_amp = peak_amp
    
    # Convert times to samples - matching original behavior
    start_pulse_samp = int(np.ceil(start_time_pulse * fs)) + 1
    peak_pulse_samp = int(np.round(peak_time_pulse * fs))
    end_pulse_samp = int(np.floor(end_time_pulse * fs))
    
    start_samp = int(np.ceil(start_time * fs)) + 1
    peak_samp = int(np.round(peak_time * fs))
    end_samp = int(np.floor(end_time * fs))
    
    pulse_interval_samp = int(np.round(pulse_interval * fs))
    
    # Create main envelope
    envelope = np.zeros(end_samp)
    
    # Rising part
    rise_points = peak_samp - start_samp + 1
    rise_time = np.arange(1, rise_points + 1)
    envelope[start_samp-1:peak_samp] = peak_amp * np.sin(0.5 * np.pi * rise_time / rise_points)
    
    # Falling part
    tc_fake = -(end_samp - peak_samp) / np.log(end_amp / (1.001 * peak_amp))
    fall_time = np.arange(1, end_samp - peak_samp + 1)
    envelope[peak_samp:end_samp] = peak_amp * np.exp(-fall_time / tc_fake)
    
    # Create pulse envelope
    pulse_envelope = np.zeros(end_pulse_samp)
    
    # Rising part of pulse
    rise_pulse_points = peak_pulse_samp - start_pulse_samp + 1
    rise_pulse_time = np.arange(1, rise_pulse_points + 1)
    pulse_envelope[start_pulse_samp-1:peak_pulse_samp] = (
        peak_amp_pulse * np.sin(0.5 * np.pi * rise_pulse_time / rise_pulse_points)
    )
    
    # Falling part of pulse
    fall_pulse_points = end_pulse_samp - peak_pulse_samp
    fall_pulse_time = np.arange(1, fall_pulse_points + 1)
    pulse_envelope[peak_pulse_samp:end_pulse_samp] = (
        peak_amp_pulse * np.cos(0.5 * np.pi * fall_pulse_time / fall_pulse_points)
    )
    
    # Create carrier signal
    phase = np.zeros(end_pulse_samp)
    carrier_time = np.arange(1, end_pulse_samp - start_pulse_samp + 2)
    phase[start_pulse_samp-1:end_pulse_samp] = np.sin(2 * np.pi * (f_fake/fs) * carrier_time)
    
    # Create single pulse
    frog_pulse = pulse_envelope * phase
    
    # Assemble complete call
    frog_call = np.zeros(end_samp)
    for i in range(pulse_count):
        start_idx = start_samp + i * pulse_interval_samp
        pulse_length = len(frog_pulse)
        end_idx = start_idx + pulse_length
        
        if end_idx > end_samp:
            pulse_portion = frog_pulse[:end_samp - start_idx]
            frog_call[start_idx-1:end_samp] = pulse_portion
        else:
            frog_call[start_idx-1:end_idx-1] = frog_pulse
    
    # Apply overall envelope
    frog_call = envelope * frog_call
    
    # Normalize
    frog_call = frog_call / np.max(np.abs(frog_call))
    
    return frog_call

In [9]:
def add_noise_normalized(signal, SNR):
    left_signal = signal.data[:,0]
    right_signal = signal.data[:,1]
    noise_gain = (10**(-SNR/20)) * np.max(np.abs(signal))
    left_noise = noise_gain * np.random.randn(len(signal))
    right_noise = noise_gain * np.random.randn(len(signal))

    left_norm_noise = left_noise / np.max(np.abs(left_noise))
    right_norm_noise = right_noise / np.max(np.abs(right_noise))
    
    # Combine with SNR scaling
    left_noisy_signal = left_signal + noise_gain * left_norm_noise
    right_noisy_signal = right_signal + noise_gain * right_norm_noise
    
    # Final normalization
    final_left_signal = left_noisy_signal / np.max(np.abs(left_noisy_signal))
    final_right_signal = right_noisy_signal / np.max(np.abs(right_noisy_signal))

    # creating the slab Binaural Instance
    final_noisy_sound = slab.Binaural([final_left_signal, final_right_signal])
    final_noisy_sound.level = level

    return final_noisy_sound

In [10]:
def create_five_pulses(pulse, n_pulses = 5, interval_duration = 1.028):
    """
        This function takes a single frog pulse and gives out 5 frog pulses (a call).

        Parameters
        pulse : the frog pulse
        n_pulses : number of pulses to add, default : 5
        interval_duration : in seconds. 
    """
    # we need to add a interpulse interval.
    left = pulse.left.data[:,0]
    right = pulse.right.data[:,0]

    samples = pulse.n_samples
    fs = pulse.samplerate
    total_samples = samples * n_pulses
    interval_samples = int(fs*interval_duration)

    combined_left = np.zeros(total_samples + (n_pulses * interval_samples))
    combined_right = np.zeros(total_samples + (n_pulses * interval_samples))

    end_point = samples
    start_point = 0
    # combining.
    for pulse in range(n_pulses):
        combined_left[start_point:end_point] = np.squeeze(left)
        combined_right[start_point:end_point] = np.squeeze(right)
        end_point = end_point + samples
        start_point = start_point + samples
        # adding inter pulse interval
        if pulse < n_pulses - 1:
            combined_left[start_point : start_point + interval_samples] = np.zeros(int(interval_samples))
            combined_right[start_point: start_point + interval_samples] = np.zeros(int(interval_samples))
            start_point = start_point + interval_samples
            end_point = start_point + samples
    
    # creating slab
    combined_5_pulses = slab.Binaural([combined_left, combined_right])

    return combined_5_pulses

In [11]:
import random

def random_values_within_range(angle, limit, num):
    """
    Generate a specified number of random integers within ±limit of angle.
    Duplicates are allowed.

    Parameters:
    angle (int): The base integer.
    limit (int): The range limit for variation.
    num (int): The number of random values to generate.

    Returns:
    list: A list of random integers within the range [angle - limit, angle + limit].
    """
    # Generate the full range of possible integer values
    possible_values = list(range(angle - limit, angle + limit + 1))
    
    # Randomly select 'num' values (with possible duplicates)
    return [random.choice(possible_values) for _ in range(num)]

In [12]:
random_values_within_range(45, 3, 10)

[47, 47, 45, 42, 42, 48, 45, 44, 47, 43]

In [13]:
def get_variable_doa_sounds(angle, variation, number_sound,SNR, frequency, name):

    angles = random_values_within_range(angle, variation, number_sound)

    frog = make_frog_call(fs=fs, frequency=frequency)
    frog_call = create_Binaural(frog)

    for idx, angle in enumerate(angles):
        left, right = get_angled_sound(angle, frog_call, False, frequency)
        noisy_right_signal = add_noise_normalized(right, SNR)
        noisy_left_signal = add_noise_normalized(left, SNR)
        noise_left_call = create_five_pulses(noisy_left_signal)
        noise_right_call = create_five_pulses(noisy_right_signal)
        noise_left_call.write(f"Stimuli/{name}/Call.{name}.{SNR}.{frequency}.{angle}.left.{idx}.wav")
        noise_right_call.write(f"Stimuli/{name}/Call.{name}.{SNR}.{frequency}.{angle}.right.{idx}.wav")

In [14]:
def get_frog_call(angle, SNR, frequency):
    frog = make_frog_call(fs=fs, frequency=frequency)
    frog_call = create_Binaural(frog)
    left, right = get_angled_sound(angle, frog_call, False, frequency)
    noisy_right_signal = add_noise_normalized(right, SNR)
    noisy_left_signal = add_noise_normalized(left, SNR)
    noisy_left_signal.write(f"Stimuli/Pulse.{SNR}.{frequency}.{angle}.left.wav")
    noisy_right_signal.write(f"Stimuli/Pulse.{SNR}.{frequency}.{angle}.right.wav")


In [15]:
def get_click_train(fs, frequency):
    clicktrain = slab.Sound.clicktrain(duration=0.472, frequency=frequency, samplerate=fs)
    clicktrain.ramp()
    return clicktrain

In [16]:
def get_filtered_noise(fs):
    noise = slab.Sound.whitenoise(duration=0.472, samplerate=fs)
    filtered_noise = noise.filter(frequency = (250, 750), kind = "bp")
    # filtered_noise = filtered_noise.ramp()
    filtered_noise.level = 80
    return filtered_noise

In [17]:
n = get_filtered_noise(fs)
n_bin = create_Binaural(n)
ss = add_noise_normalized(n_bin, 6)
ss.play()

In [18]:
def get_positive_control(angle, SNR, frequency):
    clicks = get_click_train(fs=fs, frequency=frequency)
    clicktrain = create_Binaural(clicks)
    left, right = get_angled_sound(angle, clicktrain, False, frequency)
    noisy_right_signal = add_noise_normalized(right, SNR)
    noisy_left_signal = add_noise_normalized(left, SNR)
    noisy_left_signal.write(f"Stimuli/control_positive.{SNR}.{frequency}.{angle}.left.wav")
    noisy_right_signal.write(f"Stimuli/control_positive.{SNR}.{frequency}.{angle}.right.wav")

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

def generate_control_sound(sound_type='noise', fs=44100, save_wav=False, freq = 2147):
    """
    Generate monaural control sounds: white noise or slowly modulated pure tone
    
    Parameters:
    sound_type : str
        Type of sound to generate: 'noise' or 'tone' (default: 'noise')
    fs : int
        Sampling frequency in Hz (default: 44100)
    save_wav : bool
        Whether to save the output as WAV file (default: False)
        
    Returns:
    numpy.ndarray
        1D array containing the sound samples
    """
    # Input parameters
    fsig = freq  # Signal frequency for pure tone
    nsampnom = 22646
    
    # Calculate number of samples for complete cycles
    nomcycles = (nsampnom/fs) * fsig
    actcycles = np.ceil(nomcycles)
    actnsamp = int(np.round(actcycles * fs/fsig))
    nsamp = actnsamp

    # Generate time vector
    t = np.arange(nsamp) / fs

    if sound_type == 'noise':
        # Ramped white noise parameters
        rise_fall = 0.0001  # 100 us rise/fall time
        nramp = int(np.round(rise_fall * fs))
        nongoing = nsamp - 2 * nramp
        
        # Create envelope
        onramp = np.linspace(0, 1, nramp)
        offramp = onramp[::-1]
        ongoing = np.ones(nongoing)
        envelope = np.concatenate([onramp, ongoing, offramp])
        
        # Generate and normalize white noise
        base_signal = np.random.randn(len(t))
        power = np.var(base_signal)
        signal = (1/np.sqrt(2 * power)) * base_signal
        signal = envelope * signal
        
    elif sound_type == 'tone':
        # Generate sine wave with slow rise and fall
        # Raised cosine envelope, single cycle cosine, shifted by pi
        envelope = 0.5 + 0.5 * np.cos(2 * np.pi * np.arange(nsamp)/nsamp - np.pi)
        signal = np.sin(2 * np.pi * fsig * t)
        signal = envelope * signal
        
    else:
        raise ValueError("sound_type must be either 'noise' or 'tone'")

    # Normalize final signal
    signal = signal / np.max(np.abs(signal))

    # # Plot the result
    # plt.figure(figsize=(12, 4))
    # plt.plot(signal)
    # plt.grid(True)
    # plt.title(f'{"White Noise" if sound_type == "noise" else "Pure Tone"} Control')
    # plt.xlabel('Sample')
    # plt.ylabel('Amplitude')
    # plt.show()

    # Save audio file if requested
    if save_wav:
        filename = f'control_{sound_type}.wav'
        wavfile.write(filename, fs, signal.astype(np.float32))

    return signal

# Example usage
if __name__ == "__main__":
    # Generate white noise
    noise_signal = generate_control_sound('noise')
    
    # Generate pure tone
    tone_signal = generate_control_sound('tone')

In [20]:
def get_tone(frequency):
    tone = generate_control_sound(sound_type="tone", fs=fs, freq=frequency)
    return tone

In [21]:
def get_negative_control(angle, SNR, frequency):
    tone = get_tone(frequency=frequency)
    negative_tone = create_Binaural(tone)
    left, right = get_angled_sound(angle, negative_tone, False, frequency)
    noisy_right_signal = add_noise_normalized(right, SNR)
    noisy_left_signal = add_noise_normalized(left, SNR)
    noisy_left_signal.write(f"Stimuli/control_negative.{SNR}.{frequency}.{angle}.left.wav")
    noisy_right_signal.write(f"Stimuli/control_negative.{SNR}.{frequency}.{angle}.right.wav")

# Making Sounds

In [28]:
azimuth_locations = [90, 30] # going from extreme to slight
SNRs = [6,40]
frequencies = [2147]

# 3*3*20*2 = 360 + controls / azimuth
# controls = 3 controls * 20 * channel (2) = 120
# 360 + 120 = 480

# to do
# redefine the way the code is named - should go into folders.
# remove the unique consdcsdcition.
# since if its not unique, the file names would be same, need to fix for that.


for frequency in frequencies:
    for SNR in SNRs:
        for angle in azimuth_locations:
            if angle == azimuth_locations[0]:
                print(f'Creating Angle : {angle}, SNR : {SNR}, Frequency : {frequency}')
                get_frog_call(angle=angle, SNR=SNR, frequency=frequency)
            elif angle == azimuth_locations[1]:
                print(f'Creating Angle : {angle}, SNR : {SNR}, Frequency : {frequency}')
                get_frog_call(angle=angle, SNR=SNR, frequency=frequency)
            elif angle == azimuth_locations[2]:
                print(f'Creating Angle : {angle}, SNR : {SNR}, Frequency : {frequency}')
                get_frog_call(angle=angle, SNR=SNR, frequency=frequency)
            else:
                print('something went wrong')

Creating Angle : 90, SNR : 6, Frequency : 2147
Creating Angle : 30, SNR : 6, Frequency : 2147
Creating Angle : 90, SNR : 40, Frequency : 2147
Creating Angle : 30, SNR : 40, Frequency : 2147


In [26]:
# Positive Control
azimuth_locations = [90, 30] # going from extreme to slight
SNRs = [6,40]
frequencies = [400]

for frequency in frequencies:
    for SNR in SNRs:
        for angle in azimuth_locations:
            print(f'Creating Angle : {angle}, SNR : {SNR}, Frequency : {frequency}')
            get_positive_control(angle=angle, SNR=SNR, frequency=frequency)

Creating Angle : 90, SNR : 6, Frequency : 400
Creating Angle : 30, SNR : 6, Frequency : 400
Creating Angle : 90, SNR : 40, Frequency : 400
Creating Angle : 30, SNR : 40, Frequency : 400


In [27]:
# Negative Control
azimuth_locations = [90, 30] # going from extreme to slight
SNRs = [6,40]
frequencies = [4000]

for frequency in frequencies:
    for SNR in SNRs:
        for angle in azimuth_locations:
            print(f'Creating Angle : {angle}, SNR : {SNR}, Frequency : {frequency}')
            get_negative_control(angle=angle, SNR=SNR, frequency=frequency)

Creating Angle : 90, SNR : 6, Frequency : 4000
Creating Angle : 30, SNR : 6, Frequency : 4000
Creating Angle : 90, SNR : 40, Frequency : 4000
Creating Angle : 30, SNR : 40, Frequency : 4000
